# Import model*EAU* lab data into a data-model ready sheet

In [1]:
import sqlalchemy
import pandas as pd
from lab_data import ModelEau

1. Enter the path to the file containing the data you want to import.
2. Enter the sheet name containing the raw data.

In [2]:
path = "Data/Lab/COVIDProject_Lab Measurements.xlsx"
sheet_name = "Lab analyses"

In [3]:
df = pd.read_excel(path, sheet_name=sheet_name)
df = ModelEau.clean_up(df)
samples = ModelEau.get_samples_from_lab_sheet(df)
measurements = ModelEau.get_measurements_from_lab_sheet(df)

#Preparing for excel
sample_columns_order = ["sampleID","siteID","reporterID","dateTime","dateTimeStart","dateTimeEnd","type","collection","preTreatment","pooled","children","parent","sizeL","fieldSampleTempC","shippedOnIce","storageTempC","qualityFlag","notes"]
ww_measure_order = ["uWwMeasureID", "reporterID","sampleID","labID","assayMethodID","analysisDate","reportDate","fractionAnalyzed","type","value","unit","aggregation","index","qualityFlag","accessToPublic","accessToAllOrg","accessToPHAC","accessToLocalHA","accessToProvHA","accessToOtherProv","accessToDetails","notes"]

del samples["index"]

measurements = measurements[ww_measure_order]
samples = samples[sample_columns_order]

measurements.to_excel("Data/Lab/to_paste_wwmeasure.xlsx")
samples.to_excel("Data/Lab/to_paste_sample.xlsx")

## Stop here if you are using this sheet to get excel sheets from your lab data.

Continue if you want to enter the lab data into a database

In [5]:
import odm
#preparing for DB:
measurements = measurements.rename(columns={"assayMethodID": "assayID"})
for col in ["index", "reporterID"]:
    if col in samples.columns:
        samples.drop([col],axis=1, inplace=True)
if "WwMeasureID" not in measurements.columns:
    measurements.insert(loc=0, column="WwMeasureID", value=None)

path = "Data/WBE.db"
engine = sqlalchemy.create_engine(f"sqlite:///{path}")

#Preparing for testing the "Replace" method"
engine.execute("delete from WWMeasure")
engine.execute("delete from Sample")

table_name = "WWMeasure"
odm.replace_into_db(measurements, table_name, engine)
table_name = "Sample"
odm.replace_into_db(samples, table_name, engine)